In [1]:
from pulp import *
import pandas as pd
import setup
from itertools import chain

In [ ]:
#create a class for the draft
class Draft:
    def __init__(self, draft_settings, qb_projections, rb_projections, wr_projections, te_projections, k_projections, dst_projections, adp):
        #number of teams
        #scoring settings
        #number of rounds
        #players available
        #current pick
        #class of each team

        #draft settings include number of teams, scoring settings, number of rounds
        
        self.num_teams = num_teams

        self.draft_settings = draft_settings


        self.qb_projections = qb_projections
        self.rb_projections = rb_projections
        self.wr_projections = wr_projections
        self.te_projections = te_projections
        self.k_projections = k_projections
        self.dst_projections = dst_projections

        self.adp = adp

        self.players_available = []

        #create a Team class for each team
        self.teams = []
        for i in range(self.num_teams):
            self.teams.append(Team(pick=i+1))


    def calculate_projections(self, dataset):
        #TODO
        #calculate projections for each player
        #return a list of players sorted by projections
        return dataset
    
    def calculate_vorp(self, dataset):
        #TODO
        #calculate vorp for each player
        #return a list of players sorted by vorp
        return dataset

        # dataset['VORP'] = dataset['VORP'] = dataset['Points'] - dataset['VORP']
        # return dataset.sort_values('VORP', ascending=False)



class Team:
    def __init__(self, pick):
        #list of picks
        #players drafted
        #position needs
        self.pick = pick
        

In [2]:
def maximize_vorp(players, roster_size, min_qbs, max_qbs, min_rbs, max_rbs, min_wrs, max_wrs, min_tes, max_tes,
                  starting_qb, starting_rb, starting_wr, starting_te, starting_flex, starting_sflex, picks, current_pick, adp):
    # Initialize model
    model = LpProblem("FantasyFootballOptimization", LpMaximize)
    
    # Decision Variables: Binary variables indicating if a player is selected and/or starting
    players_selected = LpVariable.dicts("PlayerSelected", players, cat='Binary')
    players_starting = LpVariable.dicts("PlayerStarting", players, cat='Binary')
    
    # Objective Function: Maximize total VORP, with a bonus for starting players
    model += lpSum([vorp[player] * players_selected[player] for player in players]) \
            + lpSum([vorp[player] * 2 * players_starting[player] for player in players]), "TotalVORP"
    
    # Constraints -- all of these roster constraints based on who's already on the roster??
    # Roster size constraint
    model += lpSum([players_selected[player] for player in players]) == roster_size
    
    # Positional constraints: Minimum and maximum number of players at each position
    model += lpSum([players_selected[player] for player in players if player_position[player] == "QB"]) >= min_qbs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "QB"]) <= max_qbs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "RB"]) >= min_rbs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "RB"]) <= max_rbs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "WR"]) >= min_wrs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "WR"]) <= max_wrs
    model += lpSum([players_selected[player] for player in players if player_position[player] == "TE"]) >= min_tes
    model += lpSum([players_selected[player] for player in players if player_position[player] == "TE"]) <= max_tes
    
    # Starting lineup constraints: Number of players at each position
    model += lpSum([players_starting[player] for player in players if player_position[player] == "QB"]) >= starting_qb
    model += lpSum([players_starting[player] for player in players if player_position[player] == "RB"]) >= starting_rb
    model += lpSum([players_starting[player] for player in players if player_position[player] == "WR"]) >= starting_wr
    model += lpSum([players_starting[player] for player in players if player_position[player] == "TE"]) >= starting_te
    # Flex position constraint
    model += lpSum([players_starting[player] for player in players if player_position[player] in ["RB", "WR", "TE"]]) == starting_flex
    # Superflex position constraint
    model += lpSum([players_starting[player] for player in players if player_position[player] in ["QB", "RB", "WR", "TE"]]) == starting_sflex
    
    # Defense and Kicker constraints -- add in eventually
    #model += lpSum([players_selected[player] for player in players if player_position[player] == "DST"]) == 1
    #model += lpSum([players_selected[player] for player in players if player_position[player] == "K"]) == 1
    

    # Constraint: Any player starting must also be selected
    for player in players:
        model += (players_starting[player]) <= (players_selected[player])
    
    # Constraint: For each pick, we can only select players with an ADP greater than or equal to the pick and not already taken
    for pick in picks:
        available_players = players[pick-picks[0]:]
        model += lpSum([players_selected[player] for player in available_players]) >= len(picks) - picks.index(pick)
        
    # Solve the problem
    model.solve()
    
    # Extract results
    selected_players = [player for player in players if players_selected[player].value() == 1]
    starting_players = [player for player in players if players_starting[player].value() == 1]
    total_vorp = value(model.objective)
    
    #df of each player and whether they are selected or starting
    player_df = pd.DataFrame(players, columns=['Player'])
    player_df['Selected'] = player_df['Player'].apply(lambda x: players_selected[x].value())
    player_df['Starting'] = player_df['Player'].apply(lambda x: players_starting[x].value())

    return selected_players, starting_players, total_vorp, player_df



In [5]:
if __name__ == "__main__":
    # Load data
    df = pd.read_csv('vorp2024.csv')
    # get into the right format
    #df['Player'] = df['FirstName'] + ' ' + df['LastName']
    players = df['Player'].tolist()
    #vorp = df.set_index('Player')['Y0_VORP'].to_dict()
    vorp = df.set_index('Player')['VORP'].to_dict()
    #player_position = df.set_index('Player')['Position'].to_dict()
    player_position = df.set_index('Player')['POS'].to_dict()
    #ADP = df.set_index('Player')['RedraftHalfPPR'].to_dict()
    ADP = df.set_index('Player')['ADP'].to_dict()

    #sort players by adp
    players = sorted(players, key=lambda x: ADP[x])
    
    # Constraints -- based on league settings
    min_qbs, max_qbs = 1, 3
    min_rbs, max_rbs = 2, 6
    min_wrs, max_wrs = 2, 6
    min_tes, max_tes = 1, 2
    starting_qb = 1
    starting_rb = 2
    starting_wr = 2
    starting_te = 1
    starting_flex = starting_rb + starting_wr + starting_te + 1
    starting_sflex = starting_qb + starting_rb + starting_wr + starting_te + starting_qb + 0
    roster_size = 13
    teams = 12

    #update draft in sleeper
    id = "1084322358149611520"
    data, player_to_index, qbs, rbs, wrs, tes, flex, names = setup.initialize_data()
    drafted_teams, drafted_pos = setup.update_draft(id, names, teams)
    drafted_players = list(chain(*drafted_teams.values()))

    #remove drafted players from available players
    available = [player for player in players if player not in drafted_players]
    

    
    def get_picks(pick, roster_size, teams):    
        picks = [pick, (2*teams+1)-pick]
        for i in range(2,roster_size):
            picks.append(picks[i-2]+(2*teams))
        return picks

    def get_team_from_pick(pick, teams): #looks complicated bc of team 12 edge cases
        if ((pick-1) // teams + 1) % 2 == 1:
            return (pick-1) % teams + 1
        else:
            return teams - ((pick-1) % teams)
        
    personal_team = 9
    personal_picks = get_picks(personal_team, roster_size, teams)

    current_pick = len(drafted_players) + 1 #needs to be based on progress of the draft
    #personal picks remaining
    personal_picks = [pick for pick in personal_picks if pick >= current_pick]

    while current_pick < personal_picks[0]:
        cur_team = get_team_from_pick(current_pick, teams)
        picks = get_picks(cur_team, roster_size, teams)

        #get players drafted by current team
        drafted_cur = drafted_teams[cur_team]
        #adjust the roster size based on the number of players drafted
        roster_size = 13 - len(drafted_cur)

        #TODO: adjust the position size based on the number of players drafted
        qb_low = max(0, min_qbs - drafted_pos[cur_team]['QB'])
        rb_low = max(0, min_rbs - drafted_pos[cur_team]['RB'])
        wr_low = max(0, min_wrs - drafted_pos[cur_team]['WR'])
        te_low = max(0, min_tes - drafted_pos[cur_team]['TE'])

        qb_high = max(0, max_qbs - drafted_pos[cur_team]['QB'])
        rb_high = max(0, max_rbs - drafted_pos[cur_team]['RB'])
        wr_high = max(0, max_wrs - drafted_pos[cur_team]['WR'])
        te_high = max(0, max_tes - drafted_pos[cur_team]['TE'])

        qb_start = max(0, starting_qb - drafted_pos[cur_team]['QB'])
        rb_start = max(0, starting_rb - drafted_pos[cur_team]['RB'])
        wr_start = max(0, starting_wr - drafted_pos[cur_team]['WR'])
        te_start = max(0, starting_te - drafted_pos[cur_team]['TE'])
        
        #TODO: run opti with adjusted positions

        #TODO: extract the model's next draft pick and add to drafted players, drafted[team]

        #TODO: update available players

        current_pick += 1

    roster_size = 13 - len(drafted_teams[cur_team])

    max_qbs = max(0, max_qbs - drafted_pos[personal_team]['QB'])
    max_rbs = max(0, max_rbs - drafted_pos[personal_team]['RB'])
    max_wrs = max(0, max_wrs - drafted_pos[personal_team]['WR'])
    max_tes = max(0, max_tes - drafted_pos[personal_team]['TE'])

    min_qbs = max(0, min_qbs - drafted_pos[personal_team]['QB'])
    min_rbs = max(0, min_rbs - drafted_pos[personal_team]['RB'])
    min_wrs = max(0, min_wrs - drafted_pos[personal_team]['WR'])
    min_tes = max(0, min_tes - drafted_pos[personal_team]['TE'])

    starting_qb = max(0, starting_qb - drafted_pos[personal_team]['QB'])
    starting_rb = max(0, starting_rb - drafted_pos[personal_team]['RB'])
    starting_wr = max(0, starting_wr - drafted_pos[personal_team]['WR'])
    starting_te = max(0, starting_te - drafted_pos[personal_team]['TE'])
    starting_flex = max(0, starting_flex - (drafted_pos[personal_team]['RB'] + drafted_pos[personal_team]['WR'] + drafted_pos[personal_team]['TE']))
    starting_sflex = max(0, starting_sflex - (drafted_pos[personal_team]['QB'] + drafted_pos[personal_team]['RB'] + drafted_pos[personal_team]['WR'] + drafted_pos[personal_team]['TE']))

    print('Roster Size:', roster_size)
    print('QB:', min_qbs, max_qbs, starting_qb)
    print('RB:', min_rbs, max_rbs, starting_rb)
    print('WR:', min_wrs, max_wrs, starting_wr)
    print('TE:', min_tes, max_tes, starting_te)
    print('Flex:', starting_flex)
    print('Superflex:', starting_sflex)
    
    # Run optimization
    selected_players, starting_players, total_vorp, player_df = maximize_vorp(available, roster_size, min_qbs, max_qbs, min_rbs, max_rbs,
                                                                   min_wrs, max_wrs, min_tes, max_tes, starting_qb, starting_rb,
                                                                   starting_wr, starting_te, starting_flex, starting_sflex, personal_picks, current_pick, ADP)
    
    # Output results, showing each pos, player, vorp, and adp sorted by adp
    selected_players = sorted(selected_players, key=lambda x: ADP[x])
    print('Selected Players')
    for player in selected_players:
        print(player, player_position[player], ADP[player])
        
    print('\nStarting Players')
    starting_players = sorted(starting_players, key=lambda x: ADP[x])
    for player in starting_players:
        print(player, player_position[player], ADP[player])
        


Roster Size: 11
QB: 1 3 1
RB: 1 5 1
WR: 1 5 1
TE: 1 2 1
Flex: 4
Superflex: 5
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/tf/0cbk24z173jdy0x9_77lgmnw0000gn/T/2b324702c6024889b2d2d7a71f19494d-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/tf/0cbk24z173jdy0x9_77lgmnw0000gn/T/2b324702c6024889b2d2d7a71f19494d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 741 COLUMNS
At line 17288 RHS
At line 18025 BOUNDS
At line 19436 ENDATA
Problem MODEL has 736 rows, 1410 columns and 12328 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1967.49 - 0.00 seconds
Cgl0004I processed model has 657 rows, 1339 columns (1339 integer (1337 of which binary)) and 10817 elements
Cbc0038I Initial state - 0 i

In [7]:
player_df[player_df['Selected']>0]

,Player,Selected,Starting
1,A.J. Brown,1.0,1.0
18,Travis Kelce,1.0,1.0
32,Patrick Mahomes,1.0,1.0
40,Joe Mixon,1.0,0.0
52,Aaron Jones,1.0,0.0
63,Tony Pollard,1.0,1.0
77,Austin Ekeler,1.0,1.0
80,Nick Chubb,1.0,0.0
96,T.J. Hockenson,1.0,0.0
146,Justin Fields,1.0,0.0


In [8]:
import pandas as pd
df = pd.read_csv('vorp2024.csv')
# get into the right format
#df['Player'] = df['FirstName'] + ' ' + df['LastName']
players = df['Player'].tolist()
#vorp = df.set_index('Player')['Y0_VORP'].to_dict()
vorp = df.set_index('Player')['VORP'].to_dict()
#player_position = df.set_index('Player')['Position'].to_dict()
player_position = df.set_index('Player')['POS'].to_dict()
#ADP = df.set_index('Player')['RedraftHalfPPR'].to_dict()
ADP = df.set_index('Player')['ADP'].to_dict()

#sort players by adp
players = sorted(players, key=lambda x: ADP[x])

In [9]:
ADP

{'Josh Allen': 31.0,
 'Jalen Hurts': 38.0,
 'Patrick Mahomes': 54.0,
 'Lamar Jackson': 41.0,
 'Justin Fields': 195.0,
 'Dak Prescott': 83.0,
 'Joe Burrow': 81.0,
 'Justin Herbert': 124.0,
 'Trevor Lawrence': 123.0,
 'Tua Tagovailoa': 113.0,
 'Jared Goff': 117.0,
 'Geno Smith': 173.0,
 'Aaron Rodgers': 144.0,
 'Russell Wilson': 212.0,
 'Jordan Love': 85.0,
 'Kirk Cousins': 120.0,
 'Deshaun Watson': 154.0,
 'Brock Purdy': 102.0,
 'Sam Howell': 268.0,
 'Matthew Stafford': 141.0,
 'Derek Carr': 185.0,
 'Daniel Jones': 203.0,
 'C.J. Stroud': 53.0,
 'Anthony Richardson': 66.0,
 'Kenny Pickett': 384.0,
 'Baker Mayfield': 150.0,
 'Bryce Young': 196.0,
 'Mac Jones': 384.0,
 'Kyler Murray': 77.0,
 'Jimmy Garoppolo': 384.0,
 'Desmond Ridder': 384.0,
 'Ryan Tannehill': 384.0,
 "Aidan O'Connell": 294.0,
 'Zach Wilson': 384.0,
 'Joshua Dobbs': 384.0,
 'Gardner Minshew': 384.0,
 'Jake Browning': 384.0,
 'Will Levis': 166.0,
 'Tommy DeVito': 384.0,
 'Taylor Heinicke': 384.0,
 'Joe Flacco': 298.0,
 'Ba